In [ ]:
 !pip install transformers

     |████████████████████████████████| 4.0 MB 6.9 MB/s 
     |████████████████████████████████| 596 kB 42.3 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 6.6 MB 35.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Load Data

In [ ]:
import pandas as pd
import torch
from google.colab import drive
drive.mount('/content/gdrive')
PATH = "/content/gdrive/My Drive/Colab Notebooks/amazon_cells_labelled.txt"
fd = pd.read_csv(PATH,sep='\t')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Mounted at /content/gdrive


In [ ]:
fd.head()

,So there is no way for me to plug it in here in the US unless I go by a converter.,0
0,"Good case, Excellent value.",1
1,Great for the jawbone.,1
2,Tied to charger for conversations lasting more...,0
3,The mic is great.,1
4,I have to jiggle the plug to get it to line up...,0


In [ ]:
fd.columns = ['sentence','value']
fd.head()

,sentence,value
0,"Good case, Excellent value.",1
1,Great for the jawbone.,1
2,Tied to charger for conversations lasting more...,0
3,The mic is great.,1
4,I have to jiggle the plug to get it to line up...,0


XLNet need [SEP] [CLS] tags at the end of each sentence

In [ ]:
sentences  = []
for sentence in fd['sentence']:
  sentence = sentence+"[SEP] [CLS]"
  sentences.append(sentence)

In [ ]:
sentences[0] ##To check if tags are added or not

'Good case, Excellent value.[SEP] [CLS]'

## Generate model


In [ ]:
from pytorch_transformers import XLNetTokenizer,XLNetForSequenceClassification

In [ ]:
from sklearn.model_selection import train_test_split
from pytorch_transformers import AdamW
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

XLNet tokenizer is used to convert our text into tokens that correspond to XLNet’s vocabulary.

In [ ]:
tokenizer  = XLNetTokenizer.from_pretrained('xlnet-base-cased',do_lower_case=True)
tokenized_text = [tokenizer.tokenize(sent) for sent in sentences]

100%|██████████| 798011/798011 [00:00<00:00, 4396357.36B/s]


In [ ]:
tokenized_text[0]

['▁good',
 '▁case',
 ',',
 '▁excellent',
 '▁value',
 '.',
 '[',
 's',
 'ep',
 ']',
 '▁[',
 'cl',
 's',
 ']']

In [ ]:
ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]
labels = fd['value'].values
print(ids[0])
print(labels[0])

[195, 363, 19, 2712, 991, 9, 10849, 23, 3882, 3158, 4145, 11974, 23, 3158]
1


In [ ]:
#Find the max len for padding the rest 
max1 = len(ids[0])
for i in ids:
  if(len(i)>max1):
    max1=len(i)
print(max1)
MAX_LEN = max1

54


In [ ]:
#Pad our sentences
input_ids2 = pad_sequences(ids,maxlen=MAX_LEN,dtype="long",truncating="post",padding="post")
print(len(input_ids2[0]))

54


In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(input_ids2,labels,test_size=0.15)
Xtrain = torch.tensor(xtrain)
Ytrain = torch.tensor(ytrain)
Xtest = torch.tensor(xtest)
Ytest = torch.tensor(ytest)

In [ ]:
batch_size = 32
train_data = TensorDataset(Xtrain.to(device),Ytrain.to(device))
test_data = TensorDataset(Xtest.to(device),Ytest.to(device))
loader = DataLoader(train_data,batch_size=batch_size)
test_loader = DataLoader(test_data,batch_size=batch_size)

In [ ]:
from transformers import XLNetForSequenceClassification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 2)
model = model.to(device)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
import torch.nn as nn
optimizer = AdamW(model.parameters(),lr=2e-5) #We pass model parameters
criterion = nn.CrossEntropyLoss()

In [ ]:
import numpy as np
def flat_accuracy(preds,labels):  #A function to predict Accuracy
  correct=0
  for i in range(0,len(labels)):
    if(preds[i]==labels[i]):
      correct+=1
  return (correct/len(labels))*100

In [ ]:
no_train = 0
epochs = 3
for epoch in range(epochs):
  model.train()
  loss1 = []
  steps = 0
  train_loss = []
  l = []
  for inputs, labels1 in loader :

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs[0],labels1)
    
    
    [train_loss.append(p.item()) for p in torch.argmax(outputs[0],axis=1).flatten() ]#our predicted 
    [l.append(z.item()) for z in labels1]# real labels
    loss.backward()
    optimizer.step()
    loss1.append(loss.item())
    no_train += inputs.size(0)
    steps += 1
  print("Current Loss is : {} Step is : {} number of Example : {} Accuracy : {}".format(loss.item(),epoch,no_train,flat_accuracy(train_loss,l)))

/usr/local/lib/python3.7/dist-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


Current Loss is : 0.6566651463508606 Step is : 0 number of Example : 849 Accuracy : 53.35689045936396
Current Loss is : 0.11069773137569427 Step is : 1 number of Example : 1698 Accuracy : 75.736160188457
Current Loss is : 0.011693504638969898 Step is : 2 number of Example : 2547 Accuracy : 92.46171967020024


In [ ]:
model.eval() #Testing our Model
acc = []
lab = []
t = 0
for inp,lab1 in test_loader:

  t+=lab1.size(0)
  outp1 = model(inp)
  [acc.append(p1.item()) for p1 in torch.argmax(outp1[0],axis=1).flatten() ]
  [lab.append(z1.item()) for z1 in lab1]
print("Total Examples : {} Accuracy {}".format(t,flat_accuracy(acc,lab)))

Total Examples : 150 Accuracy 94.0
